In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

## Titanic
data_titanic = pd.read_csv("data/titanic/train.csv")

column_titanic_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_titanic = data_titanic.rename(columns=column_titanic_dict)

train_X_titanic_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_titanic_columns = ["Survived"]
train_X_titanic_ordinal_columns = ["Sex", "Port"]

X_titanic = data_titanic[train_X_titanic_columns]
y_titanic = data_titanic[train_y_titanic_columns]
 
X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])

X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = train_test_split(X_titanic, y_titanic, test_size=0.4)
y_titanic_train = np.squeeze(y_titanic_train)
y_titanic_test = np.squeeze(y_titanic_test)

## Machine Failure
data_machine_failure = pd.read_csv("data/machine_failure/train.csv")

column_machine_failure_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_machine_failure = data_machine_failure.rename(columns=column_machine_failure_dict)

train_X_machine_failure_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_machine_failure_columns = ["Survived"]
train_X_machine_failure_ordinal_columns = ["Sex", "Port"]

X_machine_failure = data_machine_failure[train_X_machine_failure_columns]
y_machine_failure = data_machine_failure[train_y_machine_failure_columns]
 
X_machine_failure.loc[:, train_X_machine_failure_ordinal_columns] = OrdinalEncoder().fit_transform(X_machine_failure[train_X_machine_failure_ordinal_columns])

X_machine_failure_train, X_machine_failure_test, y_machine_failure_train, y_machine_failure_test = train_test_split(X_machine_failure, y_machine_failure, test_size=0.4)
y_machine_failure_train = np.squeeze(y_machine_failure_train)
y_machine_failure_test = np.squeeze(y_machine_failure_test)

## Synthetic
X_synthetic = np.random.randn(int(3e3)).reshape([int(1e3), 3])
y_synthetic = X_synthetic[:, 0] + X_synthetic[:, 1] + X_synthetic[:, 2] > 0
yn_synthetic = np.logical_not(y_synthetic)

X_synthetic[y_synthetic, :] += np.array([1, 1, 1]) * 0.1
X_synthetic[yn_synthetic, :] -= np.array([1, 1, 1]) * 0.1

X_synthetic_train, X_synthetic_test, y_synthetic_train, y_synthetic_test = train_test_split(X_synthetic, y_synthetic, test_size=0.4)
y_synthetic_train = np.squeeze(y_synthetic_train)
y_synthetic_test = np.squeeze(y_synthetic_test)

/opt/homebrew/anaconda3/envs/kaggle/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_15563/907899923.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])
/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_15563/907899923.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [10]:
## Standard Scaler

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]

logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("logr_clf", LogisticRegression()),]),
    param_grid, cv=10, verbose=0)

## Titanic
print("Titanic")
logr_clf.fit(X_titanic_train, y_titanic_train)
y_titanic_lda_clf_pred = logr_clf.predict(X_titanic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_lda_clf_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_lda_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_lda_clf_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_lda_clf_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_lda_clf_pred))
print("Best Estimator")
print(logr_clf.best_estimator_)
print("Best Parameters")
print(logr_clf.best_params_)

print("")
## Machine Failure
print("Machine Failure")
logr_clf.fit(X_machine_failure_train, y_machine_failure_train)
y_machine_failure_lda_clf_pred = logr_clf.predict(X_machine_failure_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Precision Score")
print(precision_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Recall Score")
print(recall_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("F1 Score")
print(f1_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Best Estimator")
print(logr_clf.best_estimator_)
print("Best Parameters")
print(logr_clf.best_params_)

print("")
## Synthetic
print("Synthetic")
logr_clf.fit(X_synthetic_train, y_synthetic_train)
y_synthetic_lda_clf_pred = logr_clf.predict(X_synthetic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Precision Score")
print(precision_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Recall Score")
print(recall_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("F1 Score")
print(f1_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Best Estimator")
print(logr_clf.best_estimator_)
print("Best Parameters")
print(logr_clf.best_params_)

Titanic
Confusion Matrix
[[197  32]
 [ 36  92]]
Precision Score
0.7419354838709677
Accuracy Score
0.8095238095238095
Recall Score
0.71875
F1 Score
0.7301587301587302
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('logr_clf',
                 LogisticRegression(C=0.160371874375133,
                                    multi_class='multinomial'))])
Best Parameters
{'logr_clf__C': 0.160371874375133, 'logr_clf__multi_class': 'multinomial', 'logr_clf__penalty': 'l2'}

Machine Failure
Confusion Matrix
[[191  28]
 [ 38 100]]
Precision Score
0.78125
Accuracy Score
0.8151260504201681
Recall Score
0.7246376811594203
F1 Score
0.7518796992481203
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('logr_clf', LogisticRegression(C=0.1))])
Best Parameters
{'logr_clf__C': 0.1, 'logr_clf__multi_class': 'auto', 'logr_clf__penalty': 'l2'}

Synthetic
Confusion Matrix
[[193   0]
 [  0 207]]
P

In [11]:
## Min Max Scaler

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]

logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", MinMaxScaler()),
    ("logr_clf", LogisticRegression()),]),
    param_grid, cv=10, verbose=0)

## Titanic
print("Titanic")
logr_clf.fit(X_titanic_train, y_titanic_train)
y_titanic_lda_clf_pred = logr_clf.predict(X_titanic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_lda_clf_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_lda_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_lda_clf_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_lda_clf_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_lda_clf_pred))
print("Best Estimator")
print(logr_clf.best_estimator_)
print("Best Parameters")
print(logr_clf.best_params_)

print("")
## Machine Failure
print("Machine Failure")
logr_clf.fit(X_machine_failure_train, y_machine_failure_train)
y_machine_failure_lda_clf_pred = logr_clf.predict(X_machine_failure_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Precision Score")
print(precision_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Recall Score")
print(recall_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("F1 Score")
print(f1_score(y_machine_failure_test, y_machine_failure_lda_clf_pred))
print("Best Estimator")
print(logr_clf.best_estimator_)
print("Best Parameters")
print(logr_clf.best_params_)

print("")
## Synthetic
print("Synthetic")
logr_clf.fit(X_synthetic_train, y_synthetic_train)
y_synthetic_lda_clf_pred = logr_clf.predict(X_synthetic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Precision Score")
print(precision_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Recall Score")
print(recall_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("F1 Score")
print(f1_score(y_synthetic_test, y_synthetic_lda_clf_pred))
print("Best Estimator")
print(logr_clf.best_estimator_)
print("Best Parameters")
print(logr_clf.best_params_)

Titanic
Confusion Matrix
[[195  34]
 [ 36  92]]
Precision Score
0.7301587301587301
Accuracy Score
0.803921568627451
Recall Score
0.71875
F1 Score
0.7244094488188977
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('logr_clf',
                 LogisticRegression(C=0.3257020655659783,
                                    multi_class='multinomial'))])
Best Parameters
{'logr_clf__C': 0.3257020655659783, 'logr_clf__multi_class': 'multinomial', 'logr_clf__penalty': 'l2'}

Machine Failure
Confusion Matrix
[[187  32]
 [ 37 101]]
Precision Score
0.7593984962406015
Accuracy Score
0.8067226890756303
Recall Score
0.7318840579710145
F1 Score
0.7453874538745388
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('logr_clf',
                 LogisticRegression(C=0.160371874375133,
                                    multi_class='multinomial'))])
Best Parameters
{'logr_clf__C': 0.1603718743751